In [4]:
import pandas as pd
df = pd.read_csv(r"C:\Projet 3\extracted_data_pdfplumber.csv")
df.head()


,Exp.Date:,Voyage N°:,Poids:,Montant HT:,filename,Depart,Arrivee
0,2023-12-15,PARTRP000100992,"6000,00","350,00",SupplierPreInvoiceSPI00450803.pdf,FR-77185 LOGNES\nA,FR-60300 SENLIS
1,2023-12-15,PARTRP000101238,"769,91","260,00",SupplierPreInvoiceSPI00450803.pdf,FR-95500 BONNEUIL EN\nA,FR-95130 LE PLESSIS
2,2023-12-18,PARTRP000101276,"184,62","267,50",SupplierPreInvoiceSPI00450803.pdf,FR-95500 BONNEUIL EN\nA,FR-77400 ST THIBAULT
3,2023-12-18,PARTRP000101277,"202,33","267,50",SupplierPreInvoiceSPI00450803.pdf,FR-95500 BONNEUIL EN\nA,FR-91310 MONTLHERY
4,2023-12-19,PARTRP000101305,"3160,00","350,00",SupplierPreInvoiceSPI00450803.pdf,FR-94200 IVRY SUR SEINE\nA,FR-60300 SENLIS


In [15]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import re

class GeoCoder:
    def __init__(self, user_agent="geo_locator_default"):
        self.geolocator = Nominatim(user_agent = user_agent)
        self._coordinates_cache = {} # pour un accès direct des cache (avec une complexité en O(1))

    def get_coordinates(self,location):
        if location in self._coordinates_cache :
            return self._coordinates_cache[location]  
       
        try:
            location_data = self.geolocator.geocode(location)
            if location_data : # Si on a un résultats non-null du géocodage 
                lat = location_data.latitude 
                lng = location_data.longitude
                self._coordinates_cache[location] = (lat,lng) 
                return lat,lng

            else :
                self._coordinates_cache[location] = (pd.NA,pd.NA) 
                return pd.NA, pd.NA
        except Exception as e: # capture un problème pour l'appel dans nominatim en interne ou réseau. 
                self._coordinates_cache[location]  = (pd.NA,pd.NA) # ne remonte pas le bug : continue à avoir l'info NA comme si il n'existait pas et ne bloque pas le programe par exemple
                return pd.NA, pd.NA

# calul de la distance en mile 
def calculate_distance(row): 
    if pd.notna(row['Depart_Lat']) and pd.notna(row['Arrivee_Lat']) and  pd.notna(row['Depart_Lng']) and pd.notna(row['Arrivee_Lng'])  :
       coords_1 = (row['Depart_Lat'], row['Depart_Lng'])
       coords_2 = (row['Arrivee_Lat'], row['Arrivee_Lng'])
       return geodesic(coords_1, coords_2).miles
    else :
         return pd.NA


if __name__ == '__main__': # ce block est lancé si le script est exécuté seul en mode commande
  csv_file = r"C:\Projet 3\extracted_data_pdfplumber.csv" # à remplacer par votre path correct

  try:
    df = pd.read_csv(csv_file)
     # suppression les NAN  dans la colonne Depart/Arrivee: si non supprimé cela met NaN dans pd.NA de géopy
    df.dropna(subset =['Depart', 'Arrivee'], inplace = True)
    
    #extraction  regex  avec le debut (code postale  =>   FR-\d{5} ) dans la colonne pour garder  les coordonnées de Depart et de Arrivee avec une recheche precise. Pour éviter la prise en compte des valeurs d'avant lors d'un appel sur l'api Geocode (exemple en corrigeant manuelement ou de base les mauvaises coordonnes ou chaine mal convertit avant), on se base en local sur cette valeur unique 
    df["Depart"]  = df["Depart"].str.extract(r'(FR-\d{5})' , expand=False) 
    df["Arrivee"] = df["Arrivee"].str.extract(r'(FR-\d{5})', expand=False)
        
    geo_coder_instance = GeoCoder(user_agent = "custom_geo_agent") 

       # Mise en cache  en une ligne: depart, arrivéee dans unique tableau pour avoir uniquement "une ligne" (au moment d'envoie en tache, un `map()`) en entrée, et éviter les requetes avec une "loop imbriqué" par exemple : plus simple plus lisible et donc maintenable (c'est pour éviter un gros lot de demande) .  Les entrées multiples qui peuvent exister, se servent des clés directment mis en mémoire au cas ou, grâce a l'implémentation par une  dict dans class
    all_locations_lists =  df["Depart"].to_list() + df["Arrivee"].to_list()  #  Concaténer tout en 1 liste
    locations_coord_all_dicts =  {loc:geo_coder_instance.get_coordinates(loc)    for loc in  all_locations_lists  }
  
   
  #   Assignation  des latitudes/longitudes 
    df[['Depart_Lat', 'Depart_Lng']] =   pd.DataFrame(   [  locations_coord_all_dicts.get( loc  ,(pd.NA,pd.NA))    for loc  in  df["Depart"]  ]   ,    index= df.index )
    df[['Arrivee_Lat', 'Arrivee_Lng']]  =   pd.DataFrame(    [  locations_coord_all_dicts.get(  loc ,  (pd.NA, pd.NA) )     for loc    in df["Arrivee"]  ]     , index = df.index  )

   # Calcul simple des distances 
    df['Distance'] = df.apply(calculate_distance, axis=1)


   #Gestion des Erreur sur les cas les plus generaux
  except FileNotFoundError as e :
        print (f"Attention fichier introuvable {csv_file} à changer svp !", e)
  except  pd.errors.EmptyDataError as e: 
          print (f"Attention le fichier csv ne contient pas d'information: ", e)
  except Exception as e:  
       print(f"Attention Une erreur lors de l'éxécution à était detectée , Veuillez verifier: {e} ",e)   
       raise
  

   #Verification de la variable d'ouput df avec les premières lignes si les opérations ci dessus on fonctionné (verifi des types, qu'un traitement  soit ok, ou que ce dernier affiche d'autres problèmes)
  if "df" in locals() :
       print (df.dtypes)   # vérifie l'affchage, et la console les types ou format
       print ("\nAffichage (les  premières lignes) ") #
       print (df.head(10))  
   

  # enregistrement avec une gestion plus simple. Un csv n'est pas sensé bouger a un meme endroit une fois qu'on l'enregistre)   : si y a probleme l'utilisateur le vera par le print. 
       
  try:
       df.to_csv(r'C:\Projet 3\extracted_data_with_location.csv' , index=False , encoding="utf-8") 
       print("\n le  fichier `extracted_data_with_location.csv`  est enregistré!  ")  #  message positif dans la console : permet la lisibilité de l'action par la personne
  except Exception as e:
     print(f"\n Attention un probleme de sauvegarde :", e ) # Affiche le message si une exception a lieux

Exp.Date:       object
Voyage N°:      object
Poids:          object
Montant HT:     object
filename        object
Depart          object
Arrivee         object
Depart_Lat     float64
Depart_Lng     float64
Arrivee_Lat    float64
Arrivee_Lng    float64
Distance       float64
dtype: object

Affichage (les  premières lignes) 
    Exp.Date:       Voyage N°:   Poids: Montant HT:  \
0  2023-12-15  PARTRP000100992  6000,00      350,00   
1  2023-12-15  PARTRP000101238   769,91      260,00   
2  2023-12-18  PARTRP000101276   184,62      267,50   
3  2023-12-18  PARTRP000101277   202,33      267,50   
4  2023-12-19  PARTRP000101305  3160,00      350,00   
5  2023-12-19  PARTRP000101326   292,37      256,80   
6  2023-12-19  PARTRP000101328   384,46      267,50   
7  2023-12-20  PARTRP000101365   418,11      256,80   
8  2023-12-20  PARTRP000101366   367,05      267,50   
9  2023-12-20  PARTRP000101353  1674,51      374,50   

                            filename    Depart   Arrivee  Depart_Lat

In [17]:
df_1 = pd.read_csv('extracted_data_with_location.csv')
df_1.head()

,Exp.Date:,Voyage N°:,Poids:,Montant HT:,filename,Depart,Arrivee,Depart_Lat,Depart_Lng,Arrivee_Lat,Arrivee_Lng,Distance
0,2023-12-15,PARTRP000100992,"6000,00","350,00",SupplierPreInvoiceSPI00450803.pdf,FR-77185,FR-60300,48.832504,2.638544,49.187443,2.664612,24.555845
1,2023-12-15,PARTRP000101238,"769,91","260,00",SupplierPreInvoiceSPI00450803.pdf,FR-95500,FR-95130,48.985654,2.464965,48.992035,2.224980,10.922706
2,2023-12-18,PARTRP000101276,"184,62","267,50",SupplierPreInvoiceSPI00450803.pdf,FR-95500,FR-77400,48.985654,2.464965,48.880410,2.691282,12.611706
3,2023-12-18,PARTRP000101277,"202,33","267,50",SupplierPreInvoiceSPI00450803.pdf,FR-95500,FR-91310,48.985654,2.464965,48.631851,2.268368,26.042441
4,2023-12-19,PARTRP000101305,"3160,00","350,00",SupplierPreInvoiceSPI00450803.pdf,FR-94200,FR-60300,48.815183,2.384772,49.187443,2.664612,28.698535


In [23]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import re

class GeoCoder:
    def __init__(self, user_agent="geo_locator_default"):
        self.geolocator = Nominatim(user_agent = user_agent)
        self._coordinates_cache = {}

    def get_coordinates(self,location):
       if location in self._coordinates_cache:
         return self._coordinates_cache[location]
       
       try:
           location_data = self.geolocator.geocode(location)
           if location_data :
               lat = location_data.latitude
               lng= location_data.longitude
               self._coordinates_cache[location]  = (lat,lng)
               return lat,lng
           else :
                self._coordinates_cache[location]  = (pd.NA,pd.NA)
                return pd.NA, pd.NA
       except Exception as e :
         self._coordinates_cache[location] = (pd.NA,pd.NA)
         return pd.NA,pd.NA

def calculate_distance(row):
    if pd.notna(row['Depart_Lat']) and pd.notna(row['Arrivee_Lat']) and  pd.notna(row['Depart_Lng']) and pd.notna(row['Arrivee_Lng'])  :  
        coords_1 = (row['Depart_Lat'], row['Depart_Lng'])
        coords_2 = (row['Arrivee_Lat'], row['Arrivee_Lng'])
        return geodesic(coords_1, coords_2).miles
    else:
        return pd.NA

if __name__ == '__main__': 
    csv_file = r"C:\Projet 3\extracted_data_pdfplumber.csv"  # remplacer ici
   
    try :
         df = pd.read_csv(csv_file)
        
         df.dropna(subset =['Depart', 'Arrivee'], inplace = True)

          #regex pour la suppression de texte avec des codes postaux dans chaine et uniquement : important 
         df["Depart"]  = df["Depart"].str.extract(r'(FR-\d{5})', expand = False)
         df["Arrivee"] = df["Arrivee"].str.extract(r'(FR-\d{5})' , expand = False)


         geo_coder_instance = GeoCoder(user_agent = "custom_geo_agent")
            # la map : tous  via une unique et seul ligne et via dict (voir code precedent pour memoire ou on recup /  geolocaliste, et le system de cache des variables ):  le point cle reste l'entree en parametres "list unique/variable unique : c'est  toujours la "transformation masse /par lot qu'on fait", meme quand  il faut recharger tout ou  qu'il n'y ai pas  de data. Un autre choix restes des lambdas (un by row qui execute par l'API chaque location , l'impact reste fort par l'envoie dans la phase des tests)) . C'est plus optimisé, et plus maintenable, via liste de  strings pour le code  (tous est la dans all_location): cela evite beaucoup le debugger sur plusier  elements" imbriquer lors de future manipulations "a cet endroit (par exemple) sur pandas lors d'appel vers  d'autre outils
         all_locations_lists =  df["Depart"].to_list()  +  df["Arrivee"].to_list()
         locations_coord_all_dicts =  { loc : geo_coder_instance.get_coordinates(loc)    for loc   in   all_locations_lists  } # 

        
        #  assignations variables 
         df[['Depart_Lat', 'Depart_Lng']]  = pd.DataFrame(  [locations_coord_all_dicts.get(loc ,  (pd.NA,pd.NA)  )   for loc in  df["Depart"]] , index=df.index)
         df[['Arrivee_Lat', 'Arrivee_Lng']] = pd.DataFrame( [locations_coord_all_dicts.get(loc  ,(pd.NA,pd.NA))    for loc in df['Arrivee'] ] ,index=df.index  )

         # Calcul distance simple . . le reste est par des transformation directes par pandas ou calcul sans boucle: reste beaucoup plus rapide que via un   `map(  lambdas x : ...) : . La partie Distance est obligatoire et sert lors du calcul de Consommation + Couts`

         df['Distance'] = df.apply(calculate_distance, axis = 1)


          # ajout des nouveaux champ via opération  : "convert miles -> km ", `distance (Km), ` "conso(litres), `  cout totals  (euros) ` "
         df['DistanceKM'] =  df['Distance']  * 1.60934 
         df['Conso ( litre )'] = df['DistanceKM'] * 0.33
         df['Cout total (euros)'] = df['Conso ( litre )']  * 1.80

         #Gestion d'erreur. Dans cette  partie du dev :  les "exceptions", on met toujours : FileNotFOund + Empty : la ligne doit toujours avoir son  rôle à jouer car Power bi si le code fonctionne  doit fonctionner aussi : une coherence visuel "a partir de la table  vers PowerBi  ". 

    except FileNotFoundError as e : # Message d'erreure que c'est un fichier en csv qu'est absent (important pour controler dans le workflow de Python, si la creation du  Csv depuis python   est la source d'erreur . Vous pouvez adapter votre log en fonction ) 
          print(f"Fichier CSV introuvable à modifier:{csv_file}, SVP relancez, ou mettez votre chemi", e) 
    except pd.errors.EmptyDataError as e : # un msg de logs: votre csv (qui a produit toute cette operations sur mes operations pandas a des chaines de fonctions/methode. Ca se teste en  ajoutant tous le DTypes et la  description avant un retour , meme au final via print
        print(f"Votre fichier  '{csv_file}' ,  ne contenait aucune informations !", e) # msg
    except Exception as e : 
       print (f" Un soucis a été détectée via Pandas. Veillez verifier {e}")  
       raise
   

   # partie des  transformations et "des erreurs potentielles ": l'output / le dataframe pour pouvoir a l'étape   "" PowerBI"""   de diagnostiquer d'autres eventuel problème (sur la source ) . et je vise surtout les cas numériques et la compatibilité : le but de mettre une operation pandas sur toute l'etendue est aussi une verif  pour le test du CSV en une version stable , tel  quel) : car un data analysis c'est la mise en relation data (source et ouput). C'est dans  tous le fil, ou j'essais de valider
    if "df" in locals()  :   # Si dataframe a ete créé:  on test  et affiche , pour verifié: les nouvelles col avec NaN, ou via dtype : avec un mode describe, (pour eviter les surprise dans power BI: car là , on sort en " float" au point a l'action avec Power Query via les  `to_numberic()` dans la librairie python  + .replace (que j'avais du mettre précédemment à cause de bug mais ce n'est plus necessaire, une selection precise doit fonctionner direct lors d'un simple  tri . Le NAN , ici est l'info, ou  on sait que la conversion en numberique s'est mal passee a l'etape precedentes ). La sortie console de df et dtypes a une grande importanc à mes yeux, en terme de robustesses du process sur lequel je teste tous 
   
        print(f"======== Log et Analyse  pour   : {csv_file} =======  ") # un debut ( pour un  affichage unique lors  de la data: tout le bloc de verification, avant le CSV

          # la version data_types via pandas
        print("\nData Types (de Pandas sur le DF extrait de python) \n") #
        print (df.dtypes) #   Le type d'une variables  ou formatage doit ressortir, " via le debug", des chaines ""  et non int , voir float en mode objet : cela permet dans " ce " debug "" de   verifi " qu'on controle",  de mes recommendation dans une Data (c'est le "nan  qu'on va "forcer en valeur " au debut via pandas au lieu qu'un   type chaine par exemples lors d'un read )
        print ("\n Statistiques via describe()  :\n") # la sortie en stats donne tous  en  resume + l'information et a verif via data (ou voir si  certaines données sont cohérente : des chiffres, des strings etc ..)  des col numériques et des format types : a  controlez aussi dans une étape avant PowerBI . Utile lors du débogage. Pour l'analyse sur certaines colonnes (on est  sur le max / le min si tous fait sens ou non a des moments du traitement du dataset brut  a l'export. Et donc de diagnostiqué le plus rapidemment l'etapes ou  les données ne corespondait plus avant  même  dans power BI). Il doit rester  clair. Et doit faire "reflet a toutes manipulations du dataframe dans toutes etapes , pour debugger la provenance (ex la variable  montant qui ne devrait plus faire apparaitre nan apres conversion en float en bas des col)
        print (df.describe())

          # La nouvelle partie en mode Nan qui devra s'afficher
        print (f"\n Vérifications NAN en fin  ,  pour voir le debug plus haut, en total  de toutes valeurs, par Colonnes, lors du parsing \n")# Verification "" rapide "" que tous c'est ok , l'utilisateur / dev doit pouvoir s'attendre de cette information / log sur ses colonnes qui reste dans sa visualisation dans powerBI
        print (df.isna().sum())  # ici nous cherchons si malgres tous ,  des valeurs   NaN  arrive dans un point "non prévus" pour  la colonne qu'elle reste bien  avec " 0", afin que le  graphe / select / calcul en interne "power BI" fasse que le job dans un contexte  du filtrage a la selections , des  actions  (enfin tous un workflows), des manipulations des donnés  dans un dashboards !  Vous  aussi pouvez verifié tous celas.  Le probleme et les "causes"  avec cet action de NAN /NULL doit se detecter avec l'interface d'affichage / graph : via un filtre (date et selections multiple sur d'autre categorie), et que cela donne toujours un resulta   :  si pas cas . c'est  ici que  on pourra debugger ! Et surtout si lors de mes transformation: vous n'arrivez pas à comprendre comment une chaine est passé  a du " nan" ou si mal convertit la phase amont, doit toujours  laissé  une bonne info

      # sauvegarde ,   via dataframe et test
        try : # je fais un essai via un to csv. Cette partie en python se fait bien. En général les erreur sur "read  dans des fichiers qui excedes une limite mémoire, ou  format " non correct" apparaissent dans une lecture : d'où   , la  reverification pour vous de ne pas " avoir à chercher à l'étap Power BI , dans cette dernière lecture ou l'import reste la  dernière etapes " qui valide, votre projet, par toutes mes étapes). D'où le utf-8 pour que des csv/ powerBi s'affich  au max : un message + erreur doit faire reagir la personne sur une chaine de  traitement (pas les  "dizaine /centaines  de visualisatoin en plus". Une verification simple / rapide via test par le haut (lors de "test et de feedback par le log"), est très pertinent . On doit   identifier à ce moment " le maillon faible " si jamais ce soucis (NaN / chaine et non numeric") remonte encore et via toute  transformations faites )

         df.to_csv(r"C:\Projet 3\extracted_data_with_location.csv" , encoding="utf-8" , index=False ) # utf8, sans index en début (reste coherent avec les autres fichiers/data csv si une relation multiple  est faite) : afin de valider de  mes transformation des variables du data frames qui fait transiter un csv via python et toutes mes actions. Sans aucune transformation sur les data. La manipulation de python ne change "pas l'entree"" en general lors d'import pandas : donc toutes modif  PowerBI et leurs transformation se veront si y avait " un manque  a  la  relecture en fin des traitements " en fonction des étapes faites : c'est toute la logiques en termes de " chainage de données ". Pour pouvoir aussi localiser/trouvé plus  aisement, le moindre probleme de format, types (si via  to_numeric et que vous affiche du Nan alors que tous est corrigé).   Un bon export  en python garanti le bon fonctionement a tous moments: ce qu'on attendait.    En resumer, il  vaut mieux aussi "une verification / traitement local via une donnée du debut jusqu'à la sortie". . J'espère à cet instant la vous arrivez un CSV a tous points irréprochable en amont, par " l'appel" a un `to_numeric`  qui valide toutes actions du dataframes a  ce moment (puis l'affchage) afin que PowerBI , prenne la suite a présent sans trop de souci et plus rapidement.  Le `type` reste primordial !. (toute  erreur au moment des visual : devras vous  faire reagir a cette chaine ou des  données "nan et incorrect de la base du code d'avant" n'etait pas traité  en consequence: dans un vrai projet une donnée qui se propage en "NaN " , restes bien plus problématiques , car vous ne savez pas la source !

         print (" \n fichier 'extracted_data_with_location.csv' à été enregistré correctement (utf-8) \n"  )

        except  Exception as e: 
            print (f"\n Lors de la  sauvegarde  de '{csv_file}' (avec to csv  ) une exception a été attrapé : veuillez verifié. ", e) # la partie sauvegarde est important pour ne pas avoir les probleme du   style d'ou ca  plante  via un fichier csv deja existant
            raise  # ca relève une error avec traceback,  mais l'utilisateur aura des traces a inspecté. Permet d'un dévelopeur  à déboogué



======== Log et Analyse  pour   : C:\Projet 3\extracted_data_pdfplumber.csv =======  

Data Types (de Pandas sur le DF extrait de python) 

Exp.Date:              object
Voyage N°:             object
Poids:                 object
Montant HT:            object
filename               object
Depart                 object
Arrivee                object
Depart_Lat            float64
Depart_Lng            float64
Arrivee_Lat           float64
Arrivee_Lng           float64
Distance              float64
DistanceKM            float64
Conso ( litre )       float64
Cout total (euros)    float64
dtype: object

 Statistiques via describe()  :

       Depart_Lat  Depart_Lng  Arrivee_Lat  Arrivee_Lng     Distance  \
count  797.000000  797.000000   797.000000   797.000000   797.000000   
mean    49.010394    2.523770    48.791829     2.445864    42.252112   
std      0.211179    0.307093     0.451234     4.281818   178.121396   
min     47.926260   -3.689389    45.893352  -115.457456     0.000000   
25